In [1]:
from datasets import load_dataset

# Import scikit-learn, seaborn
import seaborn as sns
import sklearn
import numpy as np
import pandas as pd
from sklearn.utils import resample
import re

# Import Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
MLMA_hate_speech = load_dataset("nedjmaou/MLMA_hate_speech")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# Convert to a DataFrame
MLMA_hate_speech_df = MLMA_hate_speech["train"].to_pandas()

In [4]:
# Find the NaN values in each column
for column in MLMA_hate_speech_df.columns:
    print(column, MLMA_hate_speech_df[column].isna().sum())

HITId 0
tweet 0
sentiment 0
directness 0
annotator_sentiment 0
target 0
group 0


In [5]:
# Find the value counts for each column
for column in MLMA_hate_speech_df.columns:
    print(column, MLMA_hate_speech_df[column].value_counts())

HITId HITId
3353    4
3352    4
3351    4
3350    4
3349    4
       ..
4542    2
4543    2
4531    2
5647    1
0       1
Name: count, Length: 5648, dtype: int64
tweet tweet
@user retard                                                                                               6
@user nigger                                                                                               6
@user twat                                                                                                 5
@user faggot                                                                                               4
@user @user retard                                                                                         4
                                                                                                          ..
sami wins. straight retarded shit                                                                          1
remembering retarded fifteen sixteen                           

In [6]:
# Get all unique values of annotator_sentiment
annotator_sentiment = MLMA_hate_speech_df["annotator_sentiment"].unique()

In [7]:
# Create a list to capture unique_annotator_sentiment
unique_annotator_sentiment = []
word_count = 0

# Use a for loop to capture each unique word in annotator_sentiment by parsing using "_"
for word in annotator_sentiment:
    words = word.split("_")
    for word in words:
        if word not in unique_annotator_sentiment:
            unique_annotator_sentiment.append(word)
            word_count += 1

unique_annotator_sentiment, word_count

(['shock', 'anger', 'confusion', 'sadness', 'indifference', 'disgust', 'fear'],
 7)

In [8]:
# Add features for each unique annotator sentiment word
for word in unique_annotator_sentiment:
    MLMA_hate_speech_df[f"{word}_annotator_sentiment"] = 0

In [9]:
MLMA_hate_speech_df.head()

,HITId,tweet,sentiment,directness,annotator_sentiment,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment
0,1,صلاة الفجر خير لك من ترديد بول البعير وسبي الن...,hateful_normal,indirect,shock,gender,individual,0,0,0,0,0,0,0
1,2,صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...,offensive,indirect,anger_confusion_sadness_indifference_disgust,other,other,0,0,0,0,0,0,0
2,3,طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...,offensive,indirect,indifference,other,individual,0,0,0,0,0,0,0
3,4,@user @user انا اوافقك بخصوص السوريين و العراق...,normal,direct,indifference,origin,other,0,0,0,0,0,0,0
4,5,هذه السعودية التي شعبها شعب الخيم و بول البعير...,normal,indirect,indifference,origin,other,0,0,0,0,0,0,0


In [10]:
# For each of the new features, encode each instance of each of the 7 words as either 0 (if not present) or 1 (if present) for the annotator sentiment
for index, row in MLMA_hate_speech_df.iterrows():
    for word in unique_annotator_sentiment:
        if word in row["annotator_sentiment"]:
            MLMA_hate_speech_df.at[index, f"{word}_annotator_sentiment"] = 1

In [11]:
MLMA_hate_speech_df.head()

,HITId,tweet,sentiment,directness,annotator_sentiment,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment
0,1,صلاة الفجر خير لك من ترديد بول البعير وسبي الن...,hateful_normal,indirect,shock,gender,individual,1,0,0,0,0,0,0
1,2,صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...,offensive,indirect,anger_confusion_sadness_indifference_disgust,other,other,0,1,1,1,1,1,0
2,3,طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...,offensive,indirect,indifference,other,individual,0,0,0,0,1,0,0
3,4,@user @user انا اوافقك بخصوص السوريين و العراق...,normal,direct,indifference,origin,other,0,0,0,0,1,0,0
4,5,هذه السعودية التي شعبها شعب الخيم و بول البعير...,normal,indirect,indifference,origin,other,0,0,0,0,1,0,0


In [12]:
# Remove the "tweet" column
MLMA_hate_speech_df.pop("tweet")

,tweet
0,صلاة الفجر خير لك من ترديد بول البعير وسبي الن...
1,صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...
2,طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...
3,@user @user انا اوافقك بخصوص السوريين و العراق...
4,هذه السعودية التي شعبها شعب الخيم و بول البعير...
...,...
18656,@user tres bien. le cinema francais c...est so...
18657,@user j’ai que des rebeus en demande d’amis aussi
18658,@user sale juif de merde
18659,@user pourquoi tant de migrants africains en e...


In [13]:
MLMA_hate_speech_df.head()

,HITId,sentiment,directness,annotator_sentiment,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment
0,1,hateful_normal,indirect,shock,gender,individual,1,0,0,0,0,0,0
1,2,offensive,indirect,anger_confusion_sadness_indifference_disgust,other,other,0,1,1,1,1,1,0
2,3,offensive,indirect,indifference,other,individual,0,0,0,0,1,0,0
3,4,normal,direct,indifference,origin,other,0,0,0,0,1,0,0
4,5,normal,indirect,indifference,origin,other,0,0,0,0,1,0,0


In [14]:
# Drop the sentiment due to redundancy
MLMA_hate_speech_df.pop("annotator_sentiment")

,annotator_sentiment
0,shock
1,anger_confusion_sadness_indifference_disgust
2,indifference
3,indifference
4,indifference
...,...
18656,sadness
18657,indifference
18658,fear_shock_indifference_sadness_disgust
18659,indifference


In [15]:
MLMA_hate_speech_df.head()

,HITId,sentiment,directness,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment
0,1,hateful_normal,indirect,gender,individual,1,0,0,0,0,0,0
1,2,offensive,indirect,other,other,0,1,1,1,1,1,0
2,3,offensive,indirect,other,individual,0,0,0,0,1,0,0
3,4,normal,direct,origin,other,0,0,0,0,1,0,0
4,5,normal,indirect,origin,other,0,0,0,0,1,0,0


In [16]:
# For sentiment, directness, target, and group, find the value counts:
for column in MLMA_hate_speech_df.columns:
    print(column, MLMA_hate_speech_df[column].value_counts())

HITId HITId
3353    4
3352    4
3351    4
3350    4
3349    4
       ..
4542    2
4543    2
4531    2
5647    1
0       1
Name: count, Length: 5648, dtype: int64
sentiment sentiment
offensive                               8194
normal                                  3058
hateful                                 1297
abusive                                  615
offensive_hateful                        554
                                        ... 
fearful_offensive_hateful                  2
abusive_hateful_disrespectful              2
fearful_offensive_disrespectful            2
abusive_hateful_disrespectful_normal       1
abusive_hateful_normal                     1
Name: count, Length: 69, dtype: int64
directness directness
indirect    13479
direct       5182
Name: count, dtype: int64
target target
origin                8039
other                 4948
disability            2356
gender                1851
sexual_orientation    1040
religion               427
Name: count, dtype: int64

In [17]:
# For sentiment, parse on _ and collect unique sentiment words
unique_sentiment = []
sentiment_word_count = 0

for word in MLMA_hate_speech_df["sentiment"].unique():
    words = word.split("_")
    for word in words:
        if word not in unique_sentiment:
            unique_sentiment.append(word)
            sentiment_word_count += 1

unique_sentiment, sentiment_word_count

(['hateful', 'normal', 'offensive', 'disrespectful', 'abusive', 'fearful'], 6)

In [18]:
# Create new columns using the unique_sentiment words
for word in unique_sentiment:
    MLMA_hate_speech_df[f"{word}_sentiment"] = 0

In [19]:
# For each entry, encode the presence of each unique sentiment word with 0 (for not present) and 1 (for present)
for index, row in MLMA_hate_speech_df.iterrows():
    for word in unique_sentiment:
        if word in row["sentiment"]:
            MLMA_hate_speech_df.at[index, f"{word}_sentiment"] = 1

In [20]:
# 0-1 encode directness
MLMA_hate_speech_df["directness"] = MLMA_hate_speech_df["directness"].map({"direct": 1, "indirect": 0})

In [21]:
MLMA_hate_speech_df.head()

,HITId,sentiment,directness,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,hateful_sentiment,normal_sentiment,offensive_sentiment,disrespectful_sentiment,abusive_sentiment,fearful_sentiment
0,1,hateful_normal,0,gender,individual,1,0,0,0,0,0,0,1,1,0,0,0,0
1,2,offensive,0,other,other,0,1,1,1,1,1,0,0,0,1,0,0,0
2,3,offensive,0,other,individual,0,0,0,0,1,0,0,0,0,1,0,0,0
3,4,normal,1,origin,other,0,0,0,0,1,0,0,0,1,0,0,0,0
4,5,normal,0,origin,other,0,0,0,0,1,0,0,0,1,0,0,0,0


In [22]:
# Drop sentiment
MLMA_hate_speech_df.pop("sentiment")

,sentiment
0,hateful_normal
1,offensive
2,offensive
3,normal
4,normal
...,...
18656,offensive
18657,disrespectful_normal
18658,offensive
18659,normal


In [23]:
MLMA_hate_speech_df.head()

,HITId,directness,target,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,hateful_sentiment,normal_sentiment,offensive_sentiment,disrespectful_sentiment,abusive_sentiment,fearful_sentiment
0,1,0,gender,individual,1,0,0,0,0,0,0,1,1,0,0,0,0
1,2,0,other,other,0,1,1,1,1,1,0,0,0,1,0,0,0
2,3,0,other,individual,0,0,0,0,1,0,0,0,0,1,0,0,0
3,4,1,origin,other,0,0,0,0,1,0,0,0,1,0,0,0,0
4,5,0,origin,other,0,0,0,0,1,0,0,0,1,0,0,0,0


In [24]:
unique_targets = MLMA_hate_speech_df["target"].unique()

In [25]:
unique_groups = MLMA_hate_speech_df["group"].unique()

In [26]:
# Combine the unique target and unique group values into a new "target_group" feature
MLMA_hate_speech_df["target_group"] = MLMA_hate_speech_df["target"].astype(str) + "_" + MLMA_hate_speech_df["group"].astype(str)

In [27]:
unique_target_groups = MLMA_hate_speech_df["target_group"].unique()
unique_target_groups

array(['gender_individual', 'other_other', 'other_individual',
       'origin_other', 'gender_women', 'other_women', 'other_arabs',
       'gender_other', 'origin_individual', 'origin_arabs',
       'other_indian/hindu', 'origin_refugees', 'other_refugees',
       'other_african_descent', 'origin_muslims', 'other_special_needs',
       'origin_women', 'religion_indian/hindu', 'religion_other',
       'origin_asians', 'other_immigrants', 'religion_individual',
       'religion_muslims', 'other_muslims', 'origin_african_descent',
       'other_jews', 'religion_christian', 'other_gay', 'other_christian',
       'religion_jews', 'origin_indian/hindu', 'gender_gay',
       'disability_special_needs', 'origin_jews', 'religion_arabs',
       'origin_gay', 'origin_left_wing_people',
       'sexual_orientation_other', 'origin_special_needs',
       'sexual_orientation_individual', 'sexual_orientation_women',
       'origin_immigrants', 'disability_women', 'gender_refugees',
       'sexual_orien

In [28]:
# Create a list of values referring to race or ethnicity
race_or_ethnicity_values = [
    "african",
    "arabs",
    "asians",
    "descent",
    "hispanics",
    "indian"
]

# Create a list of values referring to gender
gender_values = ["gender", "women"]

# Create a list of values referring to religion
religion_values = [
    "christian",
    "hindu",
    "jews",
]

# Create a list of values referring to LGBTQ
lgbtq_values = [
    "gay",
    "orientation"
]

# Create a list of values referring to nationality
nationality_values = [
    "immigrants",
    "refugees",
    "origin"
]

# Create a list of values referring to age
age_values = [
    "old"
]

# Create a list of values referring to disability
disability_values = [
    "disability", "special", "needs"
]

# Create a list of values referring to no target
none_values = ["none"]

# Create a list of values referring to class
class_values = ["wc"]

In [29]:
# 1) Normalize + tokenize once
col = "target_group"
MLMA_hate_speech_df["target_norm"] = (
    MLMA_hate_speech_df[col].astype(str).str.strip().str.lower()
)

MLMA_hate_speech_df["tokens"] = MLMA_hate_speech_df["target_norm"].apply(
    lambda s: set(filter(None, re.split(r"[_/]", s)))
)

# 2) Helper function used to to flag membership
def has_any(tokens, values):
    return int(bool(set(values) & tokens))

# 3) Create flags in one pass (no slow iterrows loops)
MLMA_hate_speech_df["race_or_ethnicity"] = MLMA_hate_speech_df["tokens"].apply(lambda t: has_any(t, race_or_ethnicity_values))
MLMA_hate_speech_df["gender"]            = MLMA_hate_speech_df["tokens"].apply(lambda t: has_any(t, gender_values))
MLMA_hate_speech_df["religion"]          = MLMA_hate_speech_df["tokens"].apply(lambda t: has_any(t, religion_values))
MLMA_hate_speech_df["lgbtq"]             = MLMA_hate_speech_df["tokens"].apply(lambda t: has_any(t, lgbtq_values))
MLMA_hate_speech_df["nationality"]       = MLMA_hate_speech_df["tokens"].apply(lambda t: has_any(t, nationality_values))
MLMA_hate_speech_df["age"]               = MLMA_hate_speech_df["tokens"].apply(lambda t: has_any(t, age_values))
MLMA_hate_speech_df["disability"]        = MLMA_hate_speech_df["tokens"].apply(lambda t: has_any(t, disability_values))
MLMA_hate_speech_df["class"]             = MLMA_hate_speech_df["tokens"].apply(lambda t: has_any(t, class_values))
MLMA_hate_speech_df["none"]              = MLMA_hate_speech_df["tokens"].apply(lambda t: has_any(t, none_values))

In [30]:
# Sanity checks
print("Sample rows containing 'religion':")
print(MLMA_hate_speech_df.loc[MLMA_hate_speech_df["target_norm"].str.contains("religion", na=False), [col, "tokens"]].head())
print("Religion==1 count:", MLMA_hate_speech_df["religion"].sum())

Sample rows containing 'religion':
              target_group                     tokens
98   religion_indian/hindu  {indian, hindu, religion}
117         religion_other          {other, religion}
152         religion_other          {other, religion}
171  religion_indian/hindu  {indian, hindu, religion}
266    religion_individual     {individual, religion}
Religion==1 count: 698


In [31]:
# Get the count of entries in the dataset
len(MLMA_hate_speech_df)

18661

In [32]:
# Get the value counts of the new columns
for col in ["race_or_ethnicity", "gender", "religion", "lgbtq", "nationality", "age", "disability", "class", "none"]:
    print(f"Value counts for {col}:")
    print(MLMA_hate_speech_df[col].value_counts())
    print()

Value counts for race_or_ethnicity:
race_or_ethnicity
0    16799
1     1862
Name: count, dtype: int64

Value counts for gender:
gender
0    15672
1     2989
Name: count, dtype: int64

Value counts for religion:
religion
0    17963
1      698
Name: count, dtype: int64

Value counts for lgbtq:
lgbtq
0    17511
1     1150
Name: count, dtype: int64

Value counts for nationality:
nationality
0    10460
1     8201
Name: count, dtype: int64

Value counts for age:
age
0    18661
Name: count, dtype: int64

Value counts for disability:
disability
0    14959
1     3702
Name: count, dtype: int64

Value counts for class:
class
0    18661
Name: count, dtype: int64

Value counts for none:
none
0    18661
Name: count, dtype: int64



In [33]:
# Drop the target column as it is no longer needed
MLMA_hate_speech_df = MLMA_hate_speech_df.drop(columns=["target"]).reset_index(drop=True)
MLMA_hate_speech_df.head()

,HITId,directness,group,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,...,tokens,race_or_ethnicity,gender,religion,lgbtq,nationality,age,disability,class,none
0,1,0,individual,1,0,0,0,0,0,0,...,"{individual, gender}",0,1,0,0,0,0,0,0,0
1,2,0,other,0,1,1,1,1,1,0,...,{other},0,0,0,0,0,0,0,0,0
2,3,0,individual,0,0,0,0,1,0,0,...,"{individual, other}",0,0,0,0,0,0,0,0,0
3,4,1,other,0,0,0,0,1,0,0,...,"{origin, other}",0,0,0,0,1,0,0,0,0
4,5,0,other,0,0,0,0,1,0,0,...,"{origin, other}",0,0,0,0,1,0,0,0,0


In [34]:
MLMA_hate_speech_df.columns

Index(['HITId', 'directness', 'group', 'shock_annotator_sentiment',
       'anger_annotator_sentiment', 'confusion_annotator_sentiment',
       'sadness_annotator_sentiment', 'indifference_annotator_sentiment',
       'disgust_annotator_sentiment', 'fear_annotator_sentiment',
       'hateful_sentiment', 'normal_sentiment', 'offensive_sentiment',
       'disrespectful_sentiment', 'abusive_sentiment', 'fearful_sentiment',
       'target_group', 'target_norm', 'tokens', 'race_or_ethnicity', 'gender',
       'religion', 'lgbtq', 'nationality', 'age', 'disability', 'class',
       'none'],
      dtype='object')

In [35]:
MLMA_hate_speech_df = MLMA_hate_speech_df.drop(columns=["target_group", "target_norm", "tokens", 'group'])

In [36]:
MLMA_hate_speech_df.head()

,HITId,directness,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,hateful_sentiment,...,fearful_sentiment,race_or_ethnicity,gender,religion,lgbtq,nationality,age,disability,class,none
0,1,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,2,0,0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [37]:
MLMA_hate_speech_df = MLMA_hate_speech_df.reset_index()

In [38]:
# Retrieve columns that contain the term "sentiment"
sentiment_columns = [col for col in MLMA_hate_speech_df.columns if "sentiment" in col and "annotator" not in col]
sentiment_columns

['hateful_sentiment',
 'normal_sentiment',
 'offensive_sentiment',
 'disrespectful_sentiment',
 'abusive_sentiment',
 'fearful_sentiment']

In [39]:
# If "abusive_sentiment", "disrespectful_sentiment", "offensive_sentiment" is 1, assign 1 to "Abusive", otherwise assign 0
MLMA_hate_speech_df["Abusive"] = MLMA_hate_speech_df[["abusive_sentiment", "disrespectful_sentiment", "offensive_sentiment"]].any(axis=1).astype(int)

# If "normal_sentiment" is 1, assign 1 to "Not Abusive", otherwise assign 0
MLMA_hate_speech_df["Not Abusive"] = MLMA_hate_speech_df["normal_sentiment"]

# If "hateful_sentiment" is 1, assign 1 to "Very Abusive"
MLMA_hate_speech_df["Very Abusive"] = MLMA_hate_speech_df["hateful_sentiment"]

# If "fearful_sentiment" is 1, assign 1 to "Very Strongly Abusive"
MLMA_hate_speech_df["Very Strongly Abusive"] = MLMA_hate_speech_df["fearful_sentiment"]

In [40]:
# Remove columns in sentiment_columns from MLMA_hate_speech_df
MLMA_hate_speech_df = MLMA_hate_speech_df.drop(columns=sentiment_columns)

In [41]:
MLMA_hate_speech_df.head()

,index,HITId,directness,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,...,lgbtq,nationality,age,disability,class,none,Abusive,Not Abusive,Very Abusive,Very Strongly Abusive
0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,1,2,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,1,0,0,0
2,2,3,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,3,4,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
4,4,5,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0


In [42]:
# Create a new "abuse_level" column
MLMA_hate_speech_df["abuse_level"] = 0

# Reassign the abuse_level column
not_abusive = MLMA_hate_speech_df[MLMA_hate_speech_df['Not Abusive'] == 1]
not_abusive['abuse_level'] = 0

abusive = MLMA_hate_speech_df[MLMA_hate_speech_df['Abusive'] == 1]
abusive['abuse_level'] = 2

very_abusive = MLMA_hate_speech_df[MLMA_hate_speech_df['Very Abusive'] == 1]
very_abusive['abuse_level'] = 3

very_strongly_abusive = MLMA_hate_speech_df[MLMA_hate_speech_df['Very Strongly Abusive'] == 1]
very_strongly_abusive['abuse_level'] = 4

# Merge all rows and sort by example_no
MLMA_hate_speech_df = pd.concat([not_abusive, abusive, very_abusive, very_strongly_abusive])
MLMA_hate_speech_df = MLMA_hate_speech_df.sort_values(by='index')

/tmp/ipython-input-1354826722.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_abusive['abuse_level'] = 0
/tmp/ipython-input-1354826722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abusive['abuse_level'] = 2
/tmp/ipython-input-1354826722.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [43]:
MLMA_hate_speech_df.drop(columns=["Abusive", "Not Abusive", "Very Abusive", "Very Strongly Abusive"])

,index,HITId,directness,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,race_or_ethnicity,gender,religion,lgbtq,nationality,age,disability,class,none,abuse_level
0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,3
1,1,2,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,2
2,2,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2
3,3,4,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18657,18657,4011,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,2
18657,18657,4011,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0
18658,18658,4012,1,1,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,2
18659,18659,4013,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0


In [44]:
MLMA_hate_speech_df.head()

,index,HITId,directness,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,...,nationality,age,disability,class,none,Abusive,Not Abusive,Very Abusive,Very Strongly Abusive,abuse_level
0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,3
1,1,2,0,0,1,1,1,1,1,0,...,0,0,0,0,0,1,0,0,0,2
2,2,3,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,2
3,3,4,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0


In [45]:
# Output MLMA_hate_speech_df as MLMA_hate_speech_full.csv
MLMA_hate_speech_df.to_csv("/content/drive/My Drive/Online MSDS/MOD C2/Political Polarization/data/MLMA_hate_speech_full.csv", index=False)

In [46]:
# Remove the index column
MLMA_hate_speech_df = MLMA_hate_speech_df.drop("index", axis=1)
MLMA_hate_speech_df

,HITId,directness,shock_annotator_sentiment,anger_annotator_sentiment,confusion_annotator_sentiment,sadness_annotator_sentiment,indifference_annotator_sentiment,disgust_annotator_sentiment,fear_annotator_sentiment,race_or_ethnicity,...,nationality,age,disability,class,none,Abusive,Not Abusive,Very Abusive,Very Strongly Abusive,abuse_level
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,3
1,2,0,0,1,1,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,2
2,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,2
3,4,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18657,4011,1,0,0,0,0,1,0,0,1,...,1,0,0,0,0,1,1,0,0,2
18657,4011,1,0,0,0,0,1,0,0,1,...,1,0,0,0,0,1,1,0,0,0
18658,4012,1,1,0,0,1,1,1,1,0,...,0,0,0,0,0,1,0,0,0,2
18659,4013,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [47]:
# Output MLMA_hate_speech_df as MLMA_hate_speech_imbalanced.csv
MLMA_hate_speech_df.to_csv("/content/drive/My Drive/Online MSDS/MOD C2/Political Polarization/data/MLMA_hate_speech_imbalanced.csv", index=False)

In [48]:
unique_target_group_indices = MLMA_hate_speech_df["abuse_level"].unique()
unique_target_group_indices

array([0, 3, 2, 4])

In [49]:
MLMA_hate_speech_df["abuse_level"].value_counts()

,count
abuse_level,
2,13637
0,5039
3,3710
4,1553


In [50]:
# Upsample the minority target groups
majority_target_group = MLMA_hate_speech_df[MLMA_hate_speech_df["abuse_level"] == 2]
majority_target_group_length = len(majority_target_group)
majority_target_group_length

13637

In [51]:
# Upsampled data
MLMA_hate_speech_df_upsampled_list = []

for current_group_index in unique_target_group_indices:
  current_group_data = MLMA_hate_speech_df[MLMA_hate_speech_df["abuse_level"] == current_group_index]

  if current_group_index == 34:
    MLMA_hate_speech_df_upsampled_list.append(current_group_data)
    continue

  # Upsample the minority class
  current_group_upsampled = resample(current_group_data, random_state=42, n_samples=majority_target_group_length, replace=True)
  MLMA_hate_speech_df_upsampled_list.append(current_group_upsampled)

In [52]:
# Combine all data in the MLMA_hate_speech_df_upsampled_list into a single dataframe
MLMA_hate_speech_df_upsampled = pd.concat([sample for sample in MLMA_hate_speech_df_upsampled_list])

In [53]:
# Remove the "Abusive", "Not Abusive", "Very Abusive", and "Very Strongly Abusive" columns
MLMA_hate_speech_df_upsampled = MLMA_hate_speech_df_upsampled.drop(columns=["Abusive", "Not Abusive", "Very Abusive", "Very Strongly Abusive"])

In [54]:
len(MLMA_hate_speech_df_upsampled), len(MLMA_hate_speech_df)

(54548, 23939)

In [55]:
MLMA_hate_speech_df_upsampled.to_csv("/content/drive/My Drive/Online MSDS/MOD C2/Political Polarization/data/MLMA_hate_speech_cleaned1.csv", index=False)